In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
from deepface import DeepFace
import pandas as pd
from sklearn.model_selection import train_test_split
import wandb
import cv2
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="Actor Detection", entity="matin-samvatian")

## Data preprocess

In [ ]:
emb = []
label = []

for i in range(3):
  for j in range(1,21):
    embedding = DeepFace.represent(img_path = f"img-input\{i}"f" ({j}).jpg", model_name = 'Facenet')
    emb.append(embedding)
    label.append(i)

In [ ]:
df = pd.DataFrame(emb)
df['label'] = label
df.to_csv('data.csv', index=False)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,category
0,0.705229,-0.467777,-0.849564,1.631884,0.781920,-1.074667,0.605650,-0.553223,0.463141,1.357267,...,0.089603,0.087537,1.491058,0.300430,-1.436116,-0.676035,0.206691,-1.134410,0.895477,0
1,-0.332985,-0.330223,-0.866652,1.594688,2.301664,0.116573,-1.522974,1.932201,1.022533,1.514163,...,-0.373162,-0.048413,0.704640,-0.371660,-2.395536,0.188537,-1.178383,0.112215,0.987411,0
2,0.290644,0.950106,-0.213829,1.623187,1.202535,-0.308924,1.210230,-0.343003,0.627491,1.614541,...,0.037902,0.677695,0.871504,-0.598366,-2.574017,-0.332315,-0.447244,-0.514461,-0.148675,0
3,-0.468002,-0.292499,-0.836633,0.629525,1.532991,-0.744017,-0.868493,0.063227,1.062264,1.181333,...,-0.227405,-0.242751,1.348131,0.502554,-2.490005,0.318960,-0.464727,-0.271058,0.054087,0
4,-0.031010,-0.418403,-0.310325,1.643237,1.757546,-0.835343,0.569866,0.098678,0.711295,1.898196,...,0.165184,0.526260,1.350363,-0.508886,-0.633738,0.203807,-0.785905,-0.245729,-0.003731,0


## load data for train/test

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
X = df.iloc[0:,0:128].values
Y = df['label'].values

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X ,Y ,test_size=0.2, random_state=24)

In [ ]:
config = wandb.config
config.learning_rate = 0.1

## Modeling

In [ ]:
model = tf.keras.models.Sequential([
    Dense(127 ,activation="relu"),
    Dense(48, activation="relu"),
    Dense(128, activation="relu"),
    Dense(3, activation="softmax") 
])

In [62]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

In [63]:
model.fit(X_train, Y_train, epochs=100, callbacks=[WandbCallback()])

Epoch 1/100
5/5 [==============================] - 1s 6ms/step - loss: 1.4052 - accuracy: 0.3459
Epoch 2/100
5/5 [==============================] - 0s 7ms/step - loss: 1.2606 - accuracy: 0.5472
Epoch 3/100
5/5 [==============================] - 0s 6ms/step - loss: 1.1121 - accuracy: 0.6101
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 0.9266 - accuracy: 0.6226
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 0.7988 - accuracy: 0.6541
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 0.6973 - accuracy: 0.6981
Epoch 7/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6609 - accuracy: 0.6792
Epoch 8/100
5/5 [==============================] - 0s 6ms/step - loss: 0.6317 - accuracy: 0.6981
Epoch 9/100
5/5 [==============================] - 0s 6ms/step - loss: 0.5984 - accuracy: 0.7358
Epoch 10/100
5/5 [==============================] - 0s 6ms/step - loss: 0.5593 - accuracy: 0.7925
Epoch 11/100
5/5 [===========

### Evaluate

In [64]:
model.evaluate(X_test, Y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.8281 - accuracy: 0.8000


[0.8280695676803589, 0.800000011920929]